# Coursera Applied Data Science Capstone Project Report
# The Battle of Neighborhoods
## By Zhemin Wu
### This is report notebook for the Applied Data Science Capstone project

## Table of Contents

### 1.  [Introduction/Business Problem](#item1)

### 2.  [Data](#item2)

### 3.  [Methodology](#item3)

### 4.  [Results](#item4)

### 5.  [Discussion](#item5)

### 6.  [Conclusion](#item6)


## Part 1: Introduction/Business Problem <a class="anchor" id="item1"></a>

## The best neighbourhood to settle down for a relocation
### Problem Description:

Assume we are a relocation service provider. One of our clients is going to relocate to another city due to some personal or business reasons, e.g relocating from Hong Kong to Shanghai. One of the most common questions he/she may ask is "Which neighborhood of that city is most comparable to my current home location and has best access to Gym/Restaurant/Theatre or less crowded/low housing cost?" Our task is just to help our client with pickcing up top choices of neighborhoods/districts in the target city according to the similarity to client's current home location and client's venue access preferences or other considerations like population density of the region.   
<img src="skyline.jpg">
### Methodology:
To complete this task, we will need to do the following:
 - Collect geospatial coordinates of client's current home and all the neighborhoods in the city they are moving to.
 - Explore the popular venues around client's current home and all the neighborhoods around the target city using Foursquare's places API
 - Specify client's venue type preferences and cluster the neighborhoods according to their prefered venue types using a Machine Learning model (e.g. K-Means).
 - Using the venues info of client's home to get similar neighborhoods in the target city, and recommond the best one/ones according to client's preferences. We can achieve this last step by doing the following
     - We can combine the venues info of client's home with those in the target city and cluster togehter with K-Means, then we can get the cluster label of client's home 
     - Get a list of all the neighborhoods in the target city with the same cluster label
     - Pick the top neighborhoods according to the client's preferences
 

## Part 2: Data <a class="anchor" id="item2"></a>

### Data Needed:
To complete this task, we will need the following data:
 - Geospatial coordinates of client's current home and all the neighborhoods in the city they are moving to.
 - Explore the popular venues around client's current home and all the neighborhoods around the target city using Foursquare's places API

### Collect geospatial coordinates data of client's current home location. The client is currently living in Central, Hong Kong
We can use geopy's Nominatim module to convert an address into latitude and longitude values. The client is currently living in the Central, Hong Kong. 

The geogspatial coordinates of client's home at Central, Hong Kong are **(22.2793278, 114.1628131)**.

### Collect base data of the target city. The client is moving to Shanghai in China
The Wikipedia page of [administrative divisions of Shanghai](https://en.wikipedia.org/wiki/List_of_administrative_divisions_of_Shanghai) contains a table listing all the 16 districts in Shanghai with the area, population and population density info. We will grab the raw table from the wiki page using the Pandas web scraping func. 

The raw dataframe looks like below

<img src="raw_data.jpg">

We will also grab a dataframe of housing cost (C¥/㎡) for each district in Shanghai from [this page](https://m.creprice.cn/city/sh.html)

The housing cost datafame looks like below. Note this table is in Chinese. It contains the rank, district name in Chinese, the average housing cost in that district and the yearly growth rate. We will combine the housing cost column from this table with the above raw dataframe based on the Chinese district name

<img src="housing_data.jpg">

### Clean up the raw base data, drop unnecessary columns and convert names
Next we can clean up the above dataframe by droping unnecessary columns and do some renaming. Note that for the "Name" column, we only need the district names. So we will convert that column also. We then insert the housing cost info from the housing cost dataframe. And we finally get the following clean dataframe.  

<img src="clean_data.jpg">

### Collect geospatial coordinates of all districts in Shanghai
We now can collect latitude and longitude values for all the 16 districts in Shanghai and combine those coordinates with the base datafram. We then obtain the follow dataframe with coordinates
<img src="clean_data_coordinates.jpg">


### Now take a look at these neighborhoods in Shanghai on a map

<img src="shanghai_districts.jpg">

### Now we can explore popular venues around all the disctricts in Shanghai and around the client's current home location in Central, Hong Kong.
We created a copy of the above dataframe and simplified it with only district name and geospatial coordinates info. We also append the same info of the client's home location in the Central, Hong Kong to the dataframe. Then we explore all the nearby venues around these locations within 5km radius using the Foursquare Places API.

We finally got a datafram looks like below (first 5 rows)
<img src="venue_data.jpg">

### So up to this point, we have completed with our data collection and we will analyze these data and build up a Machine Learning model to complete our neighborhood recommendation task


## Part 3: Methodology  <a class="anchor" id="item3"></a>
Now it's the time to analize data we prepared to perform our task. 

### Basic neighborhood info
Let's first take a look at the basic info about the the 16 districts in Shanghai from our base dataframe.

#### Area
As we can see from the below chart, Pudong and Chongming are the two largest districts in Shanghai while Hongkou and Huangpu are the two smallest districts

<img src="district_area.jpg">

#### Population Density and Housing Costs
The below chart shows the population density and housing cost of each district in Shanghai. As we can expect, due to the small sizes, Hongkou and Huangpu are the two most crowded districts in Shanghai while Huangpu is also the most expensive neighbohood for housing. Jinshan and Chongming are the two relative remote areas with lowest population densities and housing costs. One interesting thing to note that, while Pudong is the largest district in Shanghai which also has a very low population density, it is actually not very cheap for housing. The main reason is that Pudong is a newer district developed in Shanghai with some new industrial centers settled down in the region. Pudong is also Shanghai's international logistics hub where the main international airport is located. 

<img src="population_density_housing_cost.jpg">

### Exploratory data analysis on nearby venues around Shanghai and around Central, Hong Kong
Now, let's do some exploratory data analysis on the venues info data. We first grouped the venue types based on the locations by one hot encoding and calculated the corresponding appearance frequencies of each venue type for each location. We then get the below dataframe which becomes the base data set for building our machine learning models for clustering.

<img src="feature_df.jpg">

#### Let's now select some main districts in Shanghai to explore the most common nearby venue types

##### 1. Huangpu, the most crowded and expensive and busiest district in Shanghai
<img src="top_venues_huangpu.jpg">
As we can see, as the most crowded and busiet district in Shanghai, the most common venue type in Huangpu is hotel, occupying over 25% of the venues nearby. There are also a lot of restaurants, shopping malls, parks and theaters. 

##### 2. Chongming, the only remote island in Shanghai
<img src="top_venues_chongming.jpg">
As we can see, we only got two types of venues in this area, farm and Chinese restaurant. This is as expected given it's remote location and it might not be a good choice for any new comers to this city.

##### 3. Pudong, the new industrial and financial hub in Shanghai
<img src="top_venues_pudong.jpg">
As we have seen earlier, Pudong is a newly developed district in Shanghai, which is not very crowded. We can see there are also a lot of hotels and restaurants around. We can also notice that "scenic lookout" is among the top 5 most common venue types in Pudong, which gives a reason why Pudong is now the most pupular destination for the tourists to Shanghai.

##### 4. Finally, let's take a look at the client's home locataion, Central, Hong Kong
<img src="top_venues_hongkong.jpg">
Hong Kong is an internaitonal financial hub. The Central, Hong Kong is the busiest area of the city. No doubt, it will have a lot of hotels, restaurants and bars. Notice that the "Scenic Lookout" is also among the top 10 common types in the Central, Hong Kong, which is similar to the Pudong district in Shanghai. 

### Client Preference Scenarios

Now it's the time to analyze our venues data for our sttlement location recommendation task. Different clients may have different preferences of the access to the nearby venues and other considerations when choosing a place to live. Next, let's assume two different client preference scenarios and we will then analyze our data according to these scenarios and build up a model suitable to the scenario.

#### Scenario 1: The client doesn't have particular prefered venue type. The only considerations are similarity to his/her current location and if it's not very crowded.

So, for this scenario, the client only wants to choose a neighborhood which will be similar to their current home, but they don't want to move to a very crowded area. So, what we can do is to cluster the venues data of Shanghai together with the venues info of the Central, Hong Kong (which we have already combined together with our venues datafram). We can build a K-Means model to do so. And after that pick up a similar neighborhood with the lowest population density.

#### Scenario 2: The client only cares about the common daily life style venues and wants a place where they can afford buying an apartment

So, for this scenario, the client only cares about those most common daily life style venue types, including restaurants, bars, supermarkets, fitness facilities and theaters. On the other hand, the housing cost is another consideration. Thus, for this preference scenario, we can just include the relevant venue types when doing the clustering and then rank by the housing costs.

## Part 4: Results <a class="anchor" id="item4"></a>
In this section, we will adopt our methodologies for the different scenarios described above and build our machine learning models to cluster the districts in Shanghai

### Scenario 1 <a class="anchor" id="result1"></a>

As mentioned earlier, in this scenario, the client cares about the overall similarity to his/her current home and also prefer the lower crowdedmess. Thus we cluster the entire venues data with a K-Means model. The clustering results are shown below. Note in this scenario, the client's home "Central, Hong Kong" got a cluster label 1.
<img src="cluster_scen1.jpg">
As we can see from the clustering map above, we clustered the districts all around Shanghai with 4 clusters. Those districts with the red color are most similar to the client's home location in Hong Kong (cluster label 1) in terms of all nearby venue types, which are all colser to Shanghai's city center compared to other clusters. The radius of the markers represent the population density of that district. Larger marker means highly occupied region with high population density while smaller marker represents a region which is not crowded. We will disuss these results in the [Discussion Section](#item5)

### Scenario 2 <a class="anchor" id="result2"></a> 

So, for this scenario, given the client only cares about the certain daily life style venues. We will first filter our venues data with certain key words. We will then build a K-Means model to fit the filtered venue types only. On the other hand, the housing cost is another consideration. We will then look into that info also from our clustering results on the map. The clustering results are shown below. Not in this scenario, the client's home "Central, Hong Kong" got a cluster label 0.
<img src="cluster_scen2.jpg">
As we can see from the clustering map below, those districts with the purple color are most similar to the client's home location in terms of all nearby restaurants, bars, supermarkets, fitness facilities and theaters. We got 3 districts in this cluster, Qingpu, Putuo and Changning. The radius of the markers represent the housing cost of that district. Larger marker means it's more expensive to buy a house/apartment in the region. We will discuss these results in the [Discussion Section](#item5)

## Part 5: Discussion <a class="anchor" id="item5"></a>

So, up to this point, we have built up our machine learning models to cluster the 16 districts in Shanghai according to two different client preference scnenarios. Let's now take a look at these results and make our recommendations to our client.

### Scenario 1

As mentioned earlier, in this scenario, the client cares about the overall similarity to his/her current home and also prefer the lower crowdedmess. As we have seen from the [clustering results](#result1), those districts with the red color are most similar to the client's home location in Hong Kong (cluster label 1) in terms of all nearby venue types, which are all colser to Shanghai's city center compared to other clusters. Among the Cluster 1, Pudong stands out as an exceptional case which is smilar enough to our client's home but with a very low pupulation density. So it's a very good fit to our client's preference. Thus we will recommend **Pudong** to our client in this Scenario.

### Scenario 2

In this scenario, the client only cares about those most common daily life style venue types, including restaurants, bars, supermarkets, fitness facilities and theaters. The client also puts more weighting on the housing cost. As we can see from the [clustering results](#result2), those districts with the purple color are most similar to the client's home location in terms of all nearby restaurants, bars, supermarkets, fitness facilities and theaters. We got 3 districts in this cluster, Qingpu, Putuo and Changning. As we can see, among the 3 purple districts, Qingpu has the lowest housing cost. This could the the best choice for the client. However, if the client cares about the distance to the city center, it's relatively a bit too far away. For the remaining two similar choices in this cluster, Putuo and Changning have comparable housing costs while Putuo has slightly lower margin. Thus we will recommend **Qingpu** if the cliend doesn't have any preference on the distance to the city center, otherwise we will recommend **Putuo**.  

## Part 6: Conclusion <a class="anchor" id="item6"></a>

In this Capstone project, we performed a recommendation task for choosing the best settlment neighborhood for our client who's relocating from Central, Hong Kong to Shanghai. 

We first obtained some basic data from the web about the 16 districts in Shanghai, including the area, the population, population density and housing cost. We then retrieve the geospatial corrdinates of the 16 districts and combined those with the geospatial corrdinates of the Central, Hong Kong, the client's home location, in a single dataframe. We then use the Foursquare Places API to explore the common popular nearby venues around these locations. And finally, we grouped the venue types based on the locations and calculated the corresponding appearance frequencies of each venue type for each location, which then becomes the base data set for building our machine learning models for clustering.

Next, we assumed two different client preferences of the access to the nearby venues and other considerations and performed our recommendation tasks accordingly:
- **Scenario 1**: The client doesn't have particular prefered venue type. The only considerations are similarity to his/her current location and if it's not very crowded.

So, for this scenario, the client only wants to choose a neighborhood which will be similar to their current home in terms of all types of nearby venues, but they don't want to move to a very crowded area. So, we built up a K-Means model to cluster all the venues data of Shanghai together with the venues info of the Central, Hong Kong. From the clustering results, we observed that the districts sharing the same cluster label of the Central, Hong Kong are all very close to Shanghai city center. Among those, Pudong has the lowest pupulation density. Thus we recommend **Pudong** to our client in this scenario

- **Scenario 2**: The client only cares about the common daily life style venues and wants a place where they can afford buying an apartment

So, for this scenario, given the client only cares about the certain daily life style venues. We first filtered our venues data for those daily life venues. We then built a K-Means model to fit the filtered venue types only. From the clustering results, we observed 3 districts which are similar to the client's home in terms of these daily life venues. Among the 3 districts, **Qingpu** has the lowest housing cost, which is then our top recommendation to the client in this scenario. However, it's a bit far away to the Shanghai city center. If our client has concerns about the distance to the city center, then we recommend **Putuo**, which is closer to the city center but is still a cheap place for housing if compared with all other districts around. 